In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
#%load_ext line_profiler
%autoreload 2

import numpy as np
import FurgeHullam.FurgeHullam as FurgeHullam

import matplotlib.pyplot as plt
import os, glob, json, pickle
from time import perf_counter

import enterprise
from enterprise.pulsar import Pulsar

# Setup

In [ ]:
#load in data
data_pkl = './path/to/data.pkl'

with open(data_pkl, 'rb') as psr_pkl:
    psrs = pickle.load(psr_pkl)

print(len(psrs))

#only keep first n_psr pulsars
n_psr = 2
psrs = psrs[:n_psr]
print(len(psrs))
print(np.sum([len(psr.toas) for psr in psrs]))

In [ ]:
#load in noise dictionary containing pulsar noise properties
noise_json = "./path/to/noise_dictionary.json"
with open(noise_json, 'r') as f:
    noisedict = json.load(f)

In [ ]:
%%time

#set up FurgeHullam object with psr objects
FHull = FurgeHullam.FurgeHullam(psrs)

#Get total Tspan
tmin = np.min([p.toas.min() for p in psrs])
tmax = np.max([p.toas.max() for p in psrs])
Tspan = tmax - tmin

#define how high we want to go in frequency in terms of ncomp/Tspan
#this is an example for a quick test - realistically one usually wants to go up to f=1/yr, so e.g. ncomp=10 for a 10yr long dataset
ncomp = 2

#set up parameters of frequency grid - it's advisable to use 10 times ncomp grid points for accurate interpolation
fmin = 0.0
fmax = ncomp/Tspan
n_f = 10*ncomp+1

#actually do grid setup
FHull.set_up_M_N_interpolators(fmin, fmax, n_f, psrs, noisedict=noisedict)

#save grid setup to file so it can be easily loaded in later
FHull.save_N_M_to_file("N_M_freq_grid_example_maxf10perT_101comp.npz")

In [ ]:
%%time
#It's also possible to update the grid with new data if the pulsars, the epochs and noise properties stay the same
#This can be really useful for multiple realizations of simulated datasets
#And this is much faster than a full setup because Ms stay the same and only Ns change

#First we load in the grid we calculated before
FHull.load_N_M_from_file("N_M_freq_grid_example_maxf10perT_101comp.npz")

#Here we would load in new data or otherwise modify the residuals in our dataset

#update Ns only since Ms are unchanged
FHull.update_N_interpolators()

#save updated grid
FHull.save_N_M_to_file("N_M_freq_grid_example_maxf10perT_101comp_updated_w_new_data.npz")

# Fast likelihood calculation

In [ ]:
%%time
#Once we have a grid saved, we can easily and quickly setup a FurgeHullam object we can use to calculate likelihoods
FHull = FurgeHullam.FurgeHullam(psrs)
FHull.load_N_M_from_file("N_M_freq_grid_example_maxf10perT_101comp_updated_w_new_data.npz")

In [ ]:
#Let us define parameters of the CW model where we will ask for likelihoods
#Alternatively this is where an MCMC or other analysis could be setup using the FurgeHullam likelihood

################
fgw = 3e-9 #GW frequency
inc = 0.7 #inclination angle
theta = 1.4 #sky location angle
A = 5e-14 #amplitude
phase0 = 1.5 #initial Earth term phase
phi = 3.3 #other sky location angle
psi = 0.3 #polarization angle
m_c = 1e8 #chirp mass
psr_phase = 0.0 #GW phase at pulsars (in this example we assume it's the same for all pulsar)
psr_distance = 1.0 #distance to each pulsar in kpc (here we assume same for all pulsar)
################

#setup array with all the parameters in the right format
xxx = [np.cos(inc), np.cos(theta), np.log10(A), np.log10(fgw), np.log10(m_c), phase0, phi, psi]
xxx += [psr_phase, psr_distance,]*len(psrs)

xxx = np.array(xxx)
print(xxx)

#also set up an array that does not include psr phase to use with phase marginalized likelihood
xxx_nophase = [np.cos(inc), np.cos(theta), np.log10(A), np.log10(fgw), np.log10(m_c), phase0, phi, psi]
xxx_nophase += [0.0, ]*len(psrs)

xxx_nophase = np.array(xxx_nophase)
print(xxx_nophase)

#and finally set up array that only has 8 common parameters to use with phase-and-distance marginalized likelihood
xxx_com = np.copy(xxx[:8])
print(xxx_com)

In [ ]:
#calculate and time likelihood (it takes long the first time since this is when functions are compiled)
%time print(FHull.get_log_L_evolve(xxx))

In [ ]:
#calculate likelihood again (subsequent calls are fast)
%time print(FHull.get_log_L_evolve(xxx))

In [ ]:
#calculate and time phase-marginalized likelihood (it takes long the first time since this is when functions are compiled)
%time print(FHull.get_phase_marg_log_L_evolve(xxx_nophase))

In [ ]:
#calculate phase-marginalized likelihood again (subsequent calls are fast)
%time print(FHull.get_phase_marg_log_L_evolve(xxx_nophase))

In [ ]:
#calculate and time distance-and-phase-marginalized likelihood (it takes long the first time since this is when functions are compiled)
%time print(FHull.get_phase_dist_marg_log_L_evolve(xxx_com))

In [ ]:
#calculate distance-and-phase-marginalized likelihood again (subsequent calls are fast)
%time print(FHull.get_phase_dist_marg_log_L_evolve(xxx_com))